Main Code

In [2]:
import base64
import pandas as pd
from io import StringIO
from sklearn.feature_extraction.text import CountVectorizer  # bag-of-words + n-grams


def sfs_to_df(sfs_data):
    if not isinstance(sfs_data, dict) or 'content' not in sfs_data:
        return None
    content = sfs_data['content']
    if not content:
        return []
    try:
        decoded_content = base64.b64decode(content).decode('utf-8')
    except Exception:
        decoded_content = content
    csv_buffer = StringIO(decoded_content)
    df = pd.read_csv(csv_buffer)
    return df


def do_bow(documents, min_phrase_length=1, max_phrase_length=3):
    count_vectorizer = CountVectorizer(
        lowercase=True,
        stop_words='english',
        ngram_range=(min_phrase_length, max_phrase_length),
        min_df=2,
        max_df=0.95
    )
    bow_matrix = count_vectorizer.fit_transform(documents)
    feature_names = count_vectorizer.get_feature_names_out()
    return bow_matrix, feature_names


def calc_percentages(total_respondents, bow_matrix, feature_names):
    phrase_freq_pairs = {}
    for i, phrase in enumerate(feature_names):
        # Binary count non-zero entries in the bow matrix for this phrase
        phrase_freq = (bow_matrix[:, i] > 0).sum()
        phrase_freq_pairs[phrase] = phrase_freq
    phrase_percentage_pairs = []
    for phrase, count in phrase_freq_pairs.items():
        percentage = (count / total_respondents) * 100
        phrase_percentage_pair = (phrase, percentage)
        phrase_percentage_pairs.append(phrase_percentage_pair)
    return phrase_percentage_pairs


def get_top_K(K, results):
    results.sort(key=lambda x: x[1], reverse=True)
    top_k_ngrams = results[:K]
    return top_k_ngrams


def handler(input_data):
    sfs_file = input_data.get("File")
    K = input_data.get("K", 5)
    df = sfs_to_df(sfs_file)
    documents = df.iloc[:, 1]  # use col 2

    bow_matrix, feature_names = do_bow(documents, 1, 3)
    phrase_percentage_pairs = calc_percentages(len(documents), bow_matrix, feature_names)
    top_k_ngrams = get_top_K(K, phrase_percentage_pairs)
    return top_k_ngrams

Simulating node being run ...

In [3]:
import top_k_phrases_main as tkm
tkm.main("questionnaire responses.csv", 5)

Processing CSV file: questionnaire responses.csv
The top 5 Phrases are:
• ??? — 91.67% mention: clients
• ??? — 91.67% mention: managers
• ??? — 91.67% mention: need
• ??? — 83.33% mention: marketing
• ??? — 83.33% mention: sales


Testing

In [4]:
K = 5
csv_file_path = "questionnaire responses.csv"
df = pd.read_csv(csv_file_path, encoding='latin1')
documents = df.iloc[:, 1]
bow_matrix, feature_names = do_bow(documents, 1, 3)

print(bow_matrix)
feature_names

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 1019 stored elements and shape (12, 246)>
  Coords	Values
  (0, 24)	1
  (0, 94)	1
  (0, 46)	1
  (0, 33)	1
  (0, 128)	1
  (0, 173)	1
  (0, 230)	1
  (0, 140)	2
  (0, 61)	1
  (0, 0)	1
  (0, 189)	1
  (0, 123)	1
  (0, 116)	1
  (0, 59)	1
  (0, 64)	1
  (0, 200)	1
  (0, 105)	1
  (0, 197)	1
  (0, 179)	1
  (0, 30)	1
  (0, 233)	1
  (0, 19)	1
  (0, 87)	1
  (0, 113)	1
  (0, 211)	1
  :	:
  (11, 144)	1
  (11, 181)	1
  (11, 32)	1
  (11, 235)	1
  (11, 22)	1
  (11, 91)	1
  (11, 115)	1
  (11, 213)	1
  (11, 205)	1
  (11, 164)	1
  (11, 136)	1
  (11, 5)	1
  (11, 132)	1
  (11, 238)	1
  (11, 121)	1
  (11, 122)	1
  (11, 145)	1
  (11, 239)	1
  (11, 207)	1
  (11, 17)	1
  (11, 208)	1
  (11, 148)	2
  (11, 1)	2
  (11, 2)	1
  (11, 86)	1


array(['android', 'attractive', 'audience', 'audience sales',
       'audience sales marketing', 'audiences', 'awareness', 'best',
       'brands', 'buttons', 'calculator', 'calculator sales',
       'calculator sales pitch', 'calculator training',
       'calculator training design', 'case', 'case studies',
       'case studies pricing', 'case studies sales', 'cases',
       'cases industry', 'cases industry hands', 'cases industry local',
       'challenge', 'challenging', 'challenging introduce',
       'challenging introduce rcs', 'challenging iâ', 'challenging iâ ve',
       'channels', 'clear', 'clear use', 'clear use cases', 'clients',
       'clients familiar', 'clients familiar rcs', 'clients know',
       'clients meet', 'clients meet relevant', 'clients moved',
       'clients moved forwardâ', 'compare', 'compared', 'conversation',
       'conversation flows', 'conversation flows clients',
       'conversations', 'conversations clients',
       'conversations clients meet', 

In [5]:
phrase_percentage_pairs = calc_percentages(len(documents), bow_matrix, feature_names)
print(phrase_percentage_pairs)

[('android', np.float64(16.666666666666664)), ('attractive', np.float64(16.666666666666664)), ('audience', np.float64(25.0)), ('audience sales', np.float64(16.666666666666664)), ('audience sales marketing', np.float64(16.666666666666664)), ('audiences', np.float64(16.666666666666664)), ('awareness', np.float64(16.666666666666664)), ('best', np.float64(16.666666666666664)), ('brands', np.float64(16.666666666666664)), ('buttons', np.float64(16.666666666666664)), ('calculator', np.float64(50.0)), ('calculator sales', np.float64(16.666666666666664)), ('calculator sales pitch', np.float64(16.666666666666664)), ('calculator training', np.float64(16.666666666666664)), ('calculator training design', np.float64(16.666666666666664)), ('case', np.float64(66.66666666666666)), ('case studies', np.float64(58.333333333333336)), ('case studies pricing', np.float64(16.666666666666664)), ('case studies sales', np.float64(16.666666666666664)), ('cases', np.float64(66.66666666666666)), ('cases industry', 

In [6]:
phrase_percentage_pairs.sort(key=lambda x: x[1], reverse=True)
phrase_percentage_pairs

[('clients', np.float64(91.66666666666666)),
 ('managers', np.float64(91.66666666666666)),
 ('need', np.float64(91.66666666666666)),
 ('marketing', np.float64(83.33333333333334)),
 ('sales', np.float64(83.33333333333334)),
 ('clients meet', np.float64(75.0)),
 ('clients meet relevant', np.float64(75.0)),
 ('conversations', np.float64(75.0)),
 ('conversations clients', np.float64(75.0)),
 ('conversations clients meet', np.float64(75.0)),
 ('introduce', np.float64(75.0)),
 ('introduce rcs', np.float64(75.0)),
 ('introduce rcs conversations', np.float64(75.0)),
 ('meet', np.float64(75.0)),
 ('meet relevant', np.float64(75.0)),
 ('rcs conversations', np.float64(75.0)),
 ('rcs conversations clients', np.float64(75.0)),
 ('relevant', np.float64(75.0)),
 ('use', np.float64(75.0)),
 ('case', np.float64(66.66666666666666)),
 ('cases', np.float64(66.66666666666666)),
 ('cases industry', np.float64(66.66666666666666)),
 ('clear', np.float64(66.66666666666666)),
 ('clear use', np.float64(66.666666

In [7]:
top_k_ngrams = phrase_percentage_pairs[:K]
top_k_ngrams

[('clients', np.float64(91.66666666666666)),
 ('managers', np.float64(91.66666666666666)),
 ('need', np.float64(91.66666666666666)),
 ('marketing', np.float64(83.33333333333334)),
 ('sales', np.float64(83.33333333333334))]